In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# USING BDT?
USEBDT = True

In [ ]:
# Doing Run3?
ISRUN3 = False

In [ ]:
# scale to MCC8 CV?
MCC8WEIGHTS = False

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
from load_data_run123 import process_uproot as process_uproot
from load_data_run123 import process_uproot_recoveryvars as process_uproot_recoveryvars

In [ ]:
tree = "NeutrinoSelectionFilter"

BNB = ""
EXT = ""
NU  = ""
NUE = ""
DRT = ""
NCPI0 = ""
CCPI0 = ""
CCNOPI = ""
CCCPI = ""
NCNOPI = ""
NCCPI = ""

if ISRUN3:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_G1_beam_good_reco2_1e19'+ls.APPEND
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_F_G_all_reco2'+ls.APPEND
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'+ls.APPEND
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'+ls.APPEND
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'+ls.APPEND
    NCPI0  = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2'+ls.APPEND
    CCPI0  = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2'+ls.APPEND
    CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_all_run3_reco2_reco2'+ls.APPEND
    CCCPI  = 'prodgenie_filter_CCmuCPiNoPi0_overlay_mcc9_v08_00_00_33_run3_reco2_reco2'+ls.APPEND
    NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_new_run3_reco2_reco2'+ls.APPEND
    NCCPI  = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_New_run3_reco2_reco2'+ls.APPEND
    
else:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'+ls.APPEND
#    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C1_all_reco2'+ls.APPEND
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C_D_E_all_reco2'+ls.APPEND #Run1 + Run2
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'+ls.APPEND
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'+ls.APPEND
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'+ls.APPEND
    NCPI0  = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2'+ls.APPEND
    CCPI0  = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2'+ls.APPEND
    CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_all_run1_reco2_reco2'+ls.APPEND
    CCCPI  = 'prodgenie_filter_CCmuCPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'+ls.APPEND
    NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'+ls.APPEND
    NCCPI  = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'+ls.APPEND

mc = uproot.open(ls.ntuple_path+NU+".root")[ls.fold][tree]
ncpi0 = uproot.open(ls.ntuple_path+NCPI0+".root")[ls.fold][tree]
ccpi0 = uproot.open(ls.ntuple_path+CCPI0+".root")[ls.fold][tree]
ccnopi = uproot.open(ls.ntuple_path+CCNOPI+".root")[ls.fold][tree]
cccpi = uproot.open(ls.ntuple_path+CCCPI+".root")[ls.fold][tree]
ncnopi = uproot.open(ls.ntuple_path+NCNOPI+".root")[ls.fold][tree]
nccpi = uproot.open(ls.ntuple_path+NCCPI+".root")[ls.fold][tree]
nue = uproot.open(ls.ntuple_path+NUE+".root")[ls.fold][tree]
data = uproot.open(ls.ntuple_path+BNB+".root")[ls.fold][tree]
ext = uproot.open(ls.ntuple_path+EXT+".root")[ls.fold][tree]
dirt = uproot.open(ls.ntuple_path+DRT+".root")[ls.fold][tree]
lee = uproot.open(ls.ntuple_path+NUE+".root")[ls.fold][tree]

uproot_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]

variables = [
    "nu_pdg", "slpdg", "trk_score_v", "backtracked_pdg",
    "category", "ccnc",
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy0","NeutrinoEnergy1","NeutrinoEnergy2",
    "run","sub","evt",
    "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    "trk_llr_pid_score_v", # trk-PID score
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", 
    "hits_u", "hits_v", "hits_y", 
    "nproton", "mc_pdg", "slnunhits", "slnhits", "true_e_visible",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z",
    "trk_sce_start_x_v","trk_sce_start_y_v","trk_sce_start_z_v",
    "trk_sce_end_x_v","trk_sce_end_y_v","trk_sce_end_z_v",
    "trk_start_x_v","trk_start_z_v","trk_start_z_v",
    "topological_score"
]

WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune", "weightsGenie", "weightsFlux", "weightsReint"]
WEIGHTSLEE = ["weightSpline","weightTune","weightSplineTimesTune", "leeweight", "weightsGenie", "weightsFlux", "weightsReint"]
MCFVARS = ["mcf_nu_e","mcf_lep_e","mcf_actvol","mcf_nmm","mcf_nmp","mcf_nem","mcf_nep","mcf_np0","mcf_npp",
           "mcf_npm","mcf_mcshr_elec_etot","mcf_pass_ccpi0","mcf_pass_ncpi0",
           "mcf_pass_ccnopi","mcf_pass_ncnopi","mcf_pass_cccpi","mcf_pass_nccpi"]
NUEVARS = ["shr_dedx_Y", "shr_bkt_pdg", "shr_theta","shr_pfp_id_v",
           "shr_tkfit_dedx_U","shr_tkfit_dedx_V","shr_tkfit_dedx_Y",
           "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
           "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
           "shrmoliereavg","shrmoliererms",
           "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
           "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
           "trkshrhitdist2", "trkshrhitdist0","trkshrhitdist1", #distance between track and shower in 2D
           "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
           "secondshower_U_nhit","secondshower_U_vtxdist","secondshower_U_dot","secondshower_U_dir","shrclusdir0",
           "secondshower_V_nhit","secondshower_V_vtxdist","secondshower_V_dot","secondshower_V_dir","shrclusdir1",
           "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
           "shrMCSMom","DeltaRMS2h","shrPCA1CMed_5cm","CylFrac2h_1cm",
           "shr_hits_tot", "shr_hits_u_tot", "shr_hits_v_tot", "shr_hits_y_tot",
           "shr_theta_v","shr_phi_v","shr_energy_y_v",
           "shr_start_x_v","shr_start_z_v","shr_start_z_v",
           "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "trk_bkt_pdg",  
           "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
           "n_showers_contained", "shr_distance", "trk_distance",
           "matched_E", "shr_bkt_E", "trk_bkt_E",
           "shr_tkfit_nhits_Y","shr_tkfit_nhits_U","shr_tkfit_nhits_V",
           "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
           "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V",
           "trk_energy", "tksh_distance", "tksh_angle","contained_fraction",
           "shr_score", "trk_score", "trk_hits_tot","trk_len",
           "trk_hits_tot", "trk_hits_u_tot", "trk_hits_v_tot", "trk_hits_y_tot",
           "shr_energy_tot_cali", "shr_dedx_Y_cali", "trk_energy_tot","shr_id",
           "hits_ratio", "n_tracks_contained",
           "p", "pt", "selected"
]
RCVRYVARS = ["shr_energy_tot", "trk_energy_tot",
             "trk_end_x_v","trk_end_y_v","trk_end_z_v",
             "trk_phi_v","trk_theta_v","trk_len_v","trk_id",
             "shr_px","shr_py","shr_pz","shr_start_x","shr_start_y","shr_start_z","trk_hits_max",
             "shr_tkfit_dedx_u_v","shr_tkfit_dedx_v_v","shr_tkfit_dedx_y_v",
             "shr_tkfit_dedx_nhits_u_v","shr_tkfit_dedx_nhits_v_v","shr_tkfit_dedx_nhits_y_v",
             "trk2shrhitdist2","trk1trk2hitdist2","shr1shr2moliereavg","shr1trk1moliereavg","shr1trk2moliereavg",
             "trk2_id","shr2_id","trk_hits_2nd","shr_hits_2nd"
]
variables += NUEVARS
variables += RCVRYVARS

#make the list unique
variables = list(set(variables))
print(variables)

#if not ISRUN3:
variables.remove("_closestNuCosmicDist")
variables.remove("crtveto")
variables.remove("crthitpe")
    
nue = nue.pandas.df(variables + WEIGHTS, flatten=False)
mc = mc.pandas.df(variables + WEIGHTS + MCFVARS, flatten=False)
ncpi0 = ncpi0.pandas.df(variables + WEIGHTS, flatten=False)
ccpi0 = ccpi0.pandas.df(variables + WEIGHTS, flatten=False)
ccnopi = ccnopi.pandas.df(variables + WEIGHTS, flatten=False)
cccpi = cccpi.pandas.df(variables + WEIGHTS, flatten=False)
ncnopi = ncnopi.pandas.df(variables + WEIGHTS, flatten=False)
nccpi = nccpi.pandas.df(variables + WEIGHTS, flatten=False)
data = data.pandas.df(variables, flatten=False)
ext = ext.pandas.df(variables, flatten=False)
dirt = dirt.pandas.df(variables + WEIGHTS, flatten=False)
lee = lee.pandas.df(variables + WEIGHTSLEE, flatten=False)

In [ ]:
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,dirt]

for i,df in enumerate(df_v):
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    #df['weightSpline']  = df['weightSpline']  * df['weightTune']
    #df.loc[ df['npi0'] > 0, 'weightSplineTimesTune' ] = df['weightSpline'] * df['weightTune'] * 0.7 #scale down pi0s

In [ ]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!

df_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]
#df_v = [lee,mc,nue,ext,data,dirt]

for i,df in enumerate(df_v):
    up = uproot_v[i]
    process_uproot(up,df)
    process_uproot_recoveryvars(up,df)
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['anglediff_U'] = np.abs(df['secondshower_U_dir']-df['shrclusdir0'])
    df['anglediff_V'] = np.abs(df['secondshower_V_dir']-df['shrclusdir1'])
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
    df['trkthetacos'] = np.cos(df['trk_theta'])
    #df['FMdiffOBV'] = df['nu_flashmatch_score']-df['best_obviouscosmic_flashmatch_score']
    #df['FMdiff'] = df['nu_flashmatch_score']-df['best_cosmic_flashmatch_score']

In [ ]:
for i,df in enumerate(df_v):
    df.loc[df['shrmoliereavg'].isna(),'secondshower_Y_dot'] = 9999.
    #
    # secondshower_U/V/Y_dot need to be set to 9999. to match data ntuples produced with v08_00_00_43 or later;
    # the patch below is valid only to match data ntuples produced with v08_00_00_42.
    #
    df.loc[df['secondshower_U_dot'].isna(),'secondshower_U_dot'] = 0.0 #for 0p please uncomment
    df.loc[df['secondshower_V_dot'].isna(),'secondshower_V_dot'] = 0.0 #for 0p please uncomment
    df.loc[df['secondshower_Y_dot'].isna(),'secondshower_Y_dot'] = 0.0 

In [ ]:
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])
    #df['shr_tkfit_dedx_avg'] = (df['shr_tkfit_nhits_Y']*df['shr_tkfit_dedx_Y'] + df['shr_tkfit_nhits_U']*df['shr_tkfit_dedx_U'] + df['shr_tkfit_nhits_V']*df['shr_tkfit_dedx_V'])/df['shr_tkfit_nhits_tot']
    df['shr_tkfit_2cm_nhits_tot'] = (df['shr_tkfit_2cm_nhits_Y']+df['shr_tkfit_2cm_nhits_U']+df['shr_tkfit_2cm_nhits_V'])
    #df['shr_tkfit_2cm_dedx_avg'] = (df['shr_tkfit_2cm_nhits_Y']*df['shr_tkfit_2cm_dedx_Y'] + df['shr_tkfit_2cm_nhits_U']*df['shr_tkfit_2cm_dedx_U'] + df['shr_tkfit_2cm_nhits_V']*df['shr_tkfit_2cm_dedx_V'])/df['shr_tkfit_2cm_nhits_tot']
    df['shr_tkfit_gap10_nhits_tot'] = (df['shr_tkfit_gap10_nhits_Y']+df['shr_tkfit_gap10_nhits_U']+df['shr_tkfit_gap10_nhits_V'])
    #df['shr_tkfit_gap10_dedx_avg'] = (df['shr_tkfit_gap10_nhits_Y']*df['shr_tkfit_gap10_dedx_Y'] + df['shr_tkfit_gap10_nhits_U']*df['shr_tkfit_gap10_dedx_U'] + df['shr_tkfit_gap10_nhits_V']*df['shr_tkfit_gap10_dedx_V'])/df['shr_tkfit_gap10_nhits_tot']
    df.loc[:,'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_Y']
    df.loc[(df['shr_tkfit_nhits_U']>df['shr_tkfit_nhits_Y']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_U']
    df.loc[(df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_Y']) & (df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_U']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_V']

In [ ]:
for i,df in enumerate(df_v):
    df["ptOverP"] = df["pt"]/df["p"]
    df["phi1MinusPhi2"] = df["shr_phi"]-df["trk_phi"]
    df["theta1PlusTheta2"] = df["shr_theta"]+df["trk_theta"]    

In [ ]:
INTERCEPT = 0.0
SLOPE = 0.83

NMass = 0.93957
PMass = 0.93828
EMass = 0.000511

# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]
    df["reco_e_qe"] = NMass*((df["shr_energy"]+INTERCEPT)/SLOPE)/(NMass - ((df["shr_energy"]+INTERCEPT)/SLOPE)*(1-np.cos(df["shr_theta"])))
    df["reco_e_rqe"] = df["reco_e_qe"]/df["reco_e"]
    #
    df['shr_pz'] = np.cos(df['shr_theta'])*((df["shr_energy_tot_cali"]+INTERCEPT)/SLOPE)
    df['Q2'] = 2 * df['reco_e'] * (((df["shr_energy_tot_cali"]+INTERCEPT)/SLOPE) - df['shr_pz']) - EMass*EMass
    # hadronic mass
    df['Mhad'] = df['reco_e'] - ((df["shr_energy_tot_cali"]+INTERCEPT)/SLOPE)
    df['q3'] = np.sqrt( df['Mhad']*df['Mhad'] +  df['Q2'])
    # Bjorken's scaling variables
    df['Xbj'] = df['Q2'] / (2 * NMass * df['Mhad'])
    df['Ybj'] = df['Mhad'] / df['reco_e']
    # alpha_T
    df['aT'] = np.sqrt(1. - df['tksh_angle']*df['tksh_angle'])*np.sqrt((df['trk_energy']+PMass)*(df['trk_energy']+PMass)-PMass*PMass)
    
# and a way to filter out data
for i,df in enumerate(df_v):
    df["bnbdata"] = np.zeros_like(df["shr_energy"])
    df["extdata"] = np.zeros_like(df["shr_energy"])
data["bnbdata"] = np.ones_like(data["shr_energy"])
ext["extdata"] = np.ones_like(ext["shr_energy"])

# test calibrating dedx on Y plane
#data["shr_tkfit_gap10_dedx_Y"] = data["shr_tkfit_gap10_dedx_Y"]*1.03
#ext["shr_tkfit_gap10_dedx_Y"] = ext["shr_tkfit_gap10_dedx_Y"]*1.03
#data["shr_tkfit_2cm_dedx_Y"] = data["shr_tkfit_2cm_dedx_Y"]*1.03
#ext["shr_tkfit_2cm_dedx_Y"] = ext["shr_tkfit_2cm_dedx_Y"]*1.03
#data["shr_tkfit_dedx_Y"] = data["shr_tkfit_dedx_Y"]*1.03
#ext["shr_tkfit_dedx_Y"] = ext["shr_tkfit_dedx_Y"]*1.03

In [ ]:
#for i,df in enumerate(df_v):
#    df['asymm'] = np.abs(df['pi0_energy1_Y']-df['pi0_energy2_Y'])/(df['pi0_energy1_Y']+df['pi0_energy2_Y'])
#    df['pi0energy'] = 134.98 * np.sqrt( 2. / ( (1-(df['asymm'])**2) * (1-df['pi0_gammadot']) ) )

In [ ]:
# avoid double-counting of events out of FV in the NC/CC pi0 samples
# not needed anymore since we improved matching with filtered samples
#ncpi0 = ncpi0.query('category != 5')
#ccpi0 = ccpi0.query('category != 5')
#ccnopi = ccnopi.query('category != 5')
#nccpi = nccpi.query('category != 5')
#ncnopi = ncnopi.query('category != 5')

## avoid recycling unbiased ext events (i.e. selecting a slice with little nu content from these samples)
ccnopi = ccnopi.query('(nslice==0 | (slnunhits/slnhits)>0.1)')
cccpi = cccpi.query('(nslice==0 | (slnunhits/slnhits)>0.1)')
ncnopi = ncnopi.query('(nslice==0 | (slnunhits/slnhits)>0.1)')
nccpi = nccpi.query('(nslice==0 | (slnunhits/slnhits)>0.1)')

# add back the cosmic category, for background only
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.2), 'category'] = 4

In [ ]:
lee["is_signal"] = lee["category"] == 11
data["is_signal"] = data["category"] == 11
nue["is_signal"] = nue["category"] == 11
mc["is_signal"] = mc["category"] == 11
dirt["is_signal"] = dirt["category"] == 11
ext["is_signal"] = ext["category"] == 11
ncpi0["is_signal"] = ncpi0["category"] == 11
ccpi0["is_signal"] = ccpi0["category"] == 11
ccnopi["is_signal"] = ccnopi["category"] == 11
cccpi["is_signal"] = cccpi["category"] == 11
ncnopi["is_signal"] = ncnopi["category"] == 11
nccpi["is_signal"] = nccpi["category"] == 11

lee.loc[lee['category'] == 1, 'category'] = 111
lee.loc[lee['category'] == 10, 'category'] = 111
lee.loc[lee['category'] == 11, 'category'] = 111

if (USEBDT == True):
    if (ISRUN3): train_ccpi0, ccpi0 = train_test_split(ccpi0, test_size=0.5, random_state=1990)

In [ ]:
# variables to be trained on
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max","trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

LABELS = ['pi0','nonpi0']

if (USEBDT == True):
    for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            #print((ls.pickle_path+'booster_%s_0304_extnumi.model' % label), booster.best_iteration)
            #booster.save_model( (ls.pickle_path+'booster_%s_0304_extnumi.model' % label) )
            mc[label+"_score"] = booster.predict(
                xgb.DMatrix(mc[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nue[label+"_score"] = booster.predict(
                xgb.DMatrix(nue[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ext[label+"_score"] = booster.predict(
                xgb.DMatrix(ext[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            data[label+"_score"] = booster.predict(
                xgb.DMatrix(data[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            dirt[label+"_score"] = booster.predict(
                xgb.DMatrix(dirt[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            lee[label+"_score"] = booster.predict(
                xgb.DMatrix(lee[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ncpi0[label+"_score"] = booster.predict(
                xgb.DMatrix(ncpi0[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ccpi0[label+"_score"] = booster.predict(
                xgb.DMatrix(ccpi0[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ccnopi[label+"_score"] = booster.predict(
                xgb.DMatrix(ccnopi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            cccpi[label+"_score"] = booster.predict(
                xgb.DMatrix(cccpi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ncnopi[label+"_score"] = booster.predict(
                xgb.DMatrix(ncnopi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nccpi[label+"_score"] = booster.predict(
                xgb.DMatrix(nccpi[TRAINVAR]),
                ntree_limit=booster.best_iteration)

In [ ]:
# 0p BDT

TRAINVARZP = ['shrmoliereavg','shr_score', "trkfit","subcluster",
              "CosmicIPAll3D","CosmicDirAll3D",
              'secondshower_Y_nhit','secondshower_Y_vtxdist','secondshower_Y_dot','anglediff_Y',
              'secondshower_V_nhit','secondshower_V_vtxdist','secondshower_V_dot','anglediff_V',
              'secondshower_U_nhit','secondshower_U_vtxdist','secondshower_U_dot','anglediff_U',
              "shr_tkfit_2cm_dedx_U", "shr_tkfit_2cm_dedx_V", "shr_tkfit_2cm_dedx_Y",
              "shr_tkfit_gap10_dedx_U", "shr_tkfit_gap10_dedx_V", "shr_tkfit_gap10_dedx_Y",
              "shrMCSMom","DeltaRMS2h","shrPCA1CMed_5cm","CylFrac2h_1cm"]

LABELSZP = ['bkg']

if (USEBDT == True):
    for label, bkg_query in zip(LABELSZP, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi_vx.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            mc[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(mc[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            nue[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(nue[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            ext[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(ext[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            data[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(data[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            dirt[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(dirt[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            lee[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(lee[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            ncpi0[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(ncpi0[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            ccpi0[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(ccpi0[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            ccnopi[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(ccnopi[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            cccpi[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(cccpi[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            ncnopi[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(ncnopi[TRAINVARZP]),
                ntree_limit=booster.best_iteration)
            nccpi[label+"_0p_score"] = booster.predict(
                xgb.DMatrix(nccpi[TRAINVARZP]),
                ntree_limit=booster.best_iteration)

In [ ]:
dfbdt_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]

# Plotting part

In [ ]:
import plotter
import importlib
importlib.reload(plotter)

In [ ]:
samples = {
    "mc": mc,
    "nue": nue,
    "data": data,
    "ext": ext,
    "dirt": dirt,
    "ncpi0": ncpi0,
    "ccpi0": ccpi0,
    "ccnopi": ccnopi,
    "cccpi": cccpi,
    "ncnopi": ncnopi,
    "nccpi": nccpi,
    "lee": lee,
}

if ISRUN3:
    scaling = 101.0/0.943
else:
    scaling = 101.0/4.54
scaling = 1

SPLIT = 1.0
if (USEBDT == True):
    SPLIT = 2.

#0702
if ISRUN3:
    weights = {
        "mc": 7.09e-03 * scaling,
        "nue": 1.21e-04 * scaling,
        "ext": 1.06e-02 * scaling,
        "dirt": 2.95e-02 * scaling,
        "lee": 1.22e-04 * scaling,
        "ncpi0": 4.11e-03 * scaling,
        "ccpi0": 1.47e-03 * SPLIT * scaling,
        "ncnopi": 1.30e-03 * scaling,
        "nccpi": 5.25e-04 * scaling,
        "ccnopi": 1.71e-03 * scaling,
        "cccpi": 1.82e-03 * scaling,
    }
    pot = 0.943e19*scaling
else:
    weights = {
        "mc": 3.48e-02 * scaling,
        "nue": 8.65e-04 * scaling,
        "ext": 4.63e-02 * scaling, #C+D+E #C only: 1.54e-01
        "dirt": 1.42e-01 * scaling,
        "lee": 8.65e-04 * scaling,
        "ncpi0": 1.73e-02 * scaling,
        "ccpi0": 1.31e-02 * scaling,
        "ncnopi": 1.07e-02 * scaling,
        "nccpi": 5.08e-03 * scaling,
        "ccnopi": 7.81e-03 * scaling,
        "cccpi": 7.52e-03 * scaling,
    }
    pot = 4.54e19*scaling
    
my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)

In [ ]:
# muon selection
QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and trk_len > 20'
#QUERY += ' and topological_score > 0.06'
#QUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
#QUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
#QUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
#QUERY += ' and trkpid > 0.5'
#QUERY += ' and (trkthetacos < 0.853 or trkthetacos > 0.8531)'
#QUERY += ' and bkpdg == 2212 and (trkcompl < 0.5 or trkpur < 0.5)'

print (QUERY)

In [ ]:
# nue preselection
PRESQ = 'nslice == 1'
PRESQ += ' and selected == 1'
PRESQ += ' and shr_energy_tot_cali > 0.07'
PRESQ += ' and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1)'

In [ ]:
# 1eNp preselection
NPPRESQ = PRESQ
NPPRESQ += ' and n_tracks_contained > 0'

In [ ]:
# pi0-enriched selection
PIZCUTQ = PRESQ
#
PIZCUTQ += ' and CosmicIPAll3D > 30.'
PIZCUTQ += ' and CosmicDirAll3D > -0.90 and CosmicDirAll3D < 0.90'
PIZCUTQ += ' and shr_trk_sce_start_y < 100 and shr_trk_sce_end_y < 100 and shr_trk_sce_start_y > -100 and shr_trk_sce_end_y > -100'
PIZCUTQ += ' and hits_y > 50'
PIZCUTQ += ' and shr_score < 0.20'
# CC
#PIZCUTQ += ' and trk_len > 20'
#PIZCUTQ += ' and trkpid > 0.3 and n_tracks_contained > 0'
# NC
#PIZCUTQ += ' and trk_len < 20'
#PIZCUTQ += ' and (trkpid < 0.3 or n_tracks_contained == 0)'

In [ ]:
# very loose box cuts
NPVLCUTQ = NPPRESQ
NPVLCUTQ += ' and CosmicIPAll3D > 10.'
NPVLCUTQ += ' and trkpid < 0.25'
NPVLCUTQ += ' and hits_ratio > 0.5'
NPVLCUTQ += ' and trkfit < 0.90'
NPVLCUTQ += ' and n_showers_contained == 1'
NPVLCUTQ += ' and tksh_distance < 10.0'
NPVLCUTQ += ' and tksh_angle > -0.9'

In [ ]:
# loose box cuts
NPLCUTQ = NPVLCUTQ
NPLCUTQ += ' and CosmicIPAll3D > 10.'
NPLCUTQ += ' and trkpid < 0.02'
NPLCUTQ += ' and hits_ratio > 0.50'
NPLCUTQ += ' and shrmoliereavg < 9'
NPLCUTQ += ' and subcluster > 4'
NPLCUTQ += ' and trkfit < 0.65'
NPLCUTQ += ' and n_showers_contained == 1'
NPLCUTQ += ' and tksh_distance < 6.0'
NPLCUTQ += ' and (shr_tkfit_nhits_tot > 1 and shr_tkfit_dedx_max > 0.5 and shr_tkfit_dedx_max < 5.5)'
#NPLCUTQ += ' and secondshower_Y_nhit < 50'
NPLCUTQ += ' and tksh_angle > -0.9'
NPLCUTQ += ' and shr_trk_len < 300.'

In [ ]:
# tight box cuts
NPTCUTQ = NPLCUTQ
NPTCUTQ += ' and CosmicIPAll3D > 30.'
NPTCUTQ += ' and CosmicDirAll3D > -0.98 and CosmicDirAll3D < 0.98'
NPTCUTQ += ' and trkpid < 0.02'
NPTCUTQ += ' and hits_ratio > 0.65'
NPTCUTQ += ' and shr_score < 0.25'
NPTCUTQ += ' and shrmoliereavg > 2 and shrmoliereavg < 10'
NPTCUTQ += ' and subcluster > 7'
NPTCUTQ += ' and trkfit < 0.70'
NPTCUTQ += ' and n_showers_contained == 1'
NPTCUTQ += ' and tksh_distance < 4.0'
NPTCUTQ += ' and trkshrhitdist2 < 1.5'
NPTCUTQ += ' and (shr_tkfit_nhits_tot > 1 and shr_tkfit_dedx_max > 1.0 and shr_tkfit_dedx_max < 3.8)'
NPTCUTQ += ' and (secondshower_Y_nhit<=8 or secondshower_Y_dot<=0.8 or anglediff_Y<=40 or secondshower_Y_vtxdist>=100)'
#NPTCUTQ += ' and secondshower_Y_nhit < 30'
NPTCUTQ += ' and tksh_angle > -0.9 and tksh_angle < 0.70'

In [ ]:
# BDT cuts
# 0304 extnumi, pi0 and nonpi0
BDTCQ = NPLCUTQ
BDTCQ += ' and pi0_score > 0.67 and nonpi0_score > 0.70'

In [ ]:
#1e0p preselection
ZPPRESEL = PRESQ
#ZPPRESEL += ' and n_tracks_contained == 0'
ZPPRESEL += ' and n_tracks_contained > 0'
ZPPRESEL += ' and n_showers_contained > 0'

ZPBOXCUTS = ZPPRESEL
ZPBOXCUTS += ' and n_showers_contained == 1'
ZPBOXCUTS += ' and shrmoliereavg > 1 and shrmoliereavg < 8'
ZPBOXCUTS += ' and shr_score < 0.05'
ZPBOXCUTS += ' and CosmicIPAll3D > 20. '
ZPBOXCUTS += ' and (CosmicDirAll3D<0.75 and CosmicDirAll3D>-0.75)'
ZPBOXCUTS += ' and trkfit < 0.4'
ZPBOXCUTS += ' and subcluster > 6'
ZPBOXCUTS += " and (shr_tkfit_gap10_dedx_Y>1.5 & shr_tkfit_gap10_dedx_Y<2.5)"
ZPBOXCUTS += " and (shr_tkfit_gap10_dedx_U>1.5 & shr_tkfit_gap10_dedx_U<3.75)"
ZPBOXCUTS += " and (shr_tkfit_gap10_dedx_V>1.5 & shr_tkfit_gap10_dedx_V<3.75)"
ZPBOXCUTS += " and shr_tkfit_2cm_dedx_max>1. and shr_tkfit_2cm_dedx_max<4."

ZPLOOSESEL = ZPPRESEL
ZPLOOSESEL += ' and n_showers_contained == 1'
ZPLOOSESEL += ' and CosmicIPAll3D > 10.'
ZPLOOSESEL += ' and CosmicDirAll3D > -0.9 and CosmicDirAll3D < 0.9'
ZPLOOSESEL += ' and shrmoliereavg < 15'
ZPLOOSESEL += ' and subcluster > 4'
ZPLOOSESEL += ' and trkfit < 0.65'
#ZPLOOSESEL += ' and secondshower_Y_nhit < 50'
ZPLOOSESEL += ' and shr_trk_sce_start_y > -100 and shr_trk_sce_start_y < 100'
ZPLOOSESEL += ' and shr_trk_sce_end_y > -100 and shr_trk_sce_end_y < 100 '

ZPBDT = ZPLOOSESEL
ZPBDT += ' and secondshower_Y_nhit < 20'
ZPBDT += ' and bkg_0p_score >0.85'

ZPBDTLOOSE = ZPLOOSESEL
ZPBDTLOOSE += ' and secondshower_Y_nhit < 20'
ZPBDTLOOSE += ' and bkg_0p_score >0.72'

ZPBDTVLOOSE = ZPLOOSESEL
#ZPBDTVLOOSE += ' and secondshower_Y_nhit < 20'
ZPBDTVLOOSE += ' and bkg_0p_score >0.5'

In [ ]:
# Define actual selection
# QUERY = NPTCUTQ
QUERY = BDTCQ #NPPRESQ
# in case you do not want to look at the data
#QUERY += ' and bnbdata==0'
# in case you want to apply the CRT veto
#if ISRUN3: QUERY += ' and (crtveto!=1) and (_closestNuCosmicDist > 20.)'

In [ ]:
# create output files based on selection
uproot_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]
fname_v = [NUE,NU,NCPI0,CCPI0,CCNOPI,CCCPI,NCNOPI,NCCPI,NUE,EXT,BNB,DRT]

NU_Q = "~(abs(nu_pdg) == 12 & ccnc == 0)"
NU_Q += " & ~(mcf_np0==1 & mcf_nmp==0 & mcf_nmm==0 & mcf_nem==0 & mcf_nep==0)"
NU_Q += " & ~(mcf_pass_ccpi0==1)"
NU_Q += " & ~(mcf_pass_ccnopi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_ncnopi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_cccpi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_nccpi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"

for i,name in enumerate(fname_v):
    if (i == 0):
        continue # skip LEE event. This one will be done within SBNFit
        
    df = dfbdt_v[i]
    
    fout = open(ls.ntuple_path+name+'.txt','w')
    dfsel = df.query(QUERY)
    if (i == 1):
        dfsel = dfsel.query(NU_Q)
    print ('file %s has %i selected entries'%(name,dfsel.shape[0]))
    for i,row in dfsel.iterrows():
        run = row['run']
        sub = row['sub']
        evt = row['evt']
        fout.write('%i %i %i \n'%(run,sub,evt))
    fout.close()

In [ ]:
# v = "$\cos\left(\theta_{\mu}\right)$"

#QUERY += " & pi0truth_elec_etot<15 & npi0 == 0 & ccnc == 0"
#QUERY += " & muon_e>0.3 & npi0 == 0 & ccnc == 0"
#QUERY = "(muon_e<0.3 | pi0truth_elec_etot>=15) & npi0 == 0 & npion == 0 & ccnc == 0 & nu_pdg == 14 & category == 2"
#QUERY += " & ~((muon_e<0.3 | pi0truth_elec_etot>=15) & npi0 == 0 & npion == 0 & ccnc == 0 & nu_pdg == 14 & category != 5)"


VARIABLE, BINS, RANGE, XTIT = 'reco_e', 14, (0.15,1.55), r"Reconstructed Energy [GeV]"
# VARIABLE, BINS, RANGE, XTIT = 'reco',25,(0,1),r"topological score"
# VARIABLE, BINS, RANGE, XTIT = 'pi0_mass_Y',20,(0,500),"pi0 mass Y"

#VARIABLE, BINS, RANGE, XTIT = 'muon_e',20,(0.,1.),r"Muon Energy [GeV]"
#VARIABLE, BINS, RANGE, XTIT = 'pi0truth_elec_etot',20,(0.,60),r"Michel Energy [GeV]"

#fig, ax1, ax2, a, b, c = my_plotter.plot_variable(
fig, ax1, ax2 = my_plotter.plot_variable(
    VARIABLE,   
    query=QUERY,#+" and category != 111",
    kind="event_category",
    #kind="sample",
    #kind="interaction",
    #kind='particle_pdg',
    draw_sys=True,
    stacksort=3,
    title=XTIT,
    #bins=asymm_bins,
    bins=BINS,
    range=RANGE,
)[0:3]

print("Profile likelihood: %.2f sigma @ 1.01e21 POT" % my_plotter.significance_likelihood)
print("s/sqrt(b): %.2f sigma @ 1.01e21 POT" % my_plotter.significance)

#print(sum(a[0][-2])+sum(c))

#ax1.set_ylim(0,40)
#ax1.set_yscale("log")
#ax1.set_ylim(0,240)
ax1.set_ylim(0,ax1.get_ylim()[1]*1.25)
#ax2.set_ylim(0.5,1.5)
#fig.savefig(ls.plots_path+VARIABLE+"_"+date_time+".pdf")

In [ ]:
from scipy.stats import poisson
from matplotlib import pyplot as plt

In [ ]:
# studies showing agreement between data and MC stats (w/ confidence intervals) vs. cut value
# thruth-cut on MC sample to avoid double counting with truth-filters (i.e. ccpi0, ncpi0, cccpi, ...)
NU_Q = "~(abs(nu_pdg) == 12 & ccnc == 0)"
NU_Q += " & ~(mcf_np0==1 & mcf_nmp==0 & mcf_nmm==0 & mcf_nem==0 & mcf_nep==0)"
NU_Q += " & ~(mcf_pass_ccpi0==1)"
NU_Q += " & ~(mcf_pass_ccnopi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_ncnopi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_cccpi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
NU_Q += " & ~(mcf_pass_nccpi==1 & (nslice==0 | (slnunhits/slnhits)>0.1))"
def GetNumEvents(VAR,EMIN,EMAX,QUERY):
    TOT  = 0.
    DATA = 0.
    for key, sample in samples.items():
        THISQUERY = QUERY
        # calculate weight for sample
        #print (THISQUERY)
        weight = 1
        if (key != 'data'):
            weight = weights[key]    
        if (key == 'mc'):
            THISQUERY += ' and %s'%NU_Q
        # calculate bare entries
        dfsub = sample.query(THISQUERY)
        vals_v = dfsub[VAR].values
        weights_v = np.ones(len(vals_v)) * weight
        if ((key != 'data') and (key != 'ext')):
            weights_v = dfsub['weightSplineTimesTune'].values * weight
        v,be = np.histogram(vals_v,bins=np.array([EMIN,EMAX]),weights=weights_v)
        #print ('sample %s has %i entries'%(key,v[0]))
        if (key == 'data'):
            DATA += v[0]
        else:
            TOT += v[0]
    print ('DATA : %.01f. EXPECTATION : %.01f'%(DATA,TOT))
    # poisson interval
    # 68%
    range68 = poisson.interval(0.68,TOT)
    range95 = poisson.interval(0.95,TOT)
    range99 = poisson.interval(0.99,TOT)
    #print ('range 68%% %.02f -- %.02f'%(range68[0],range68[1]))
    #print ('range 95%% %.02f -- %.02f'%(range95[0],range95[1]))
    return DATA,TOT,range68[0],range68[1],range95[0],range95[1],range99[0],range99[1]
#BDTCUT_V = np.concatenate((np.linspace(0.0,0.1,21),np.linspace(0.5,1.0,15)),axis=0)
BDTCUT_V = np.linspace(0.0,0.7,14)
#BDTCU
DATA_V = []
EXP_V = []
MIN68_V = []
MAX68_V = []
MIN95_V = []
MAX95_V = []
MIN99_V = []
MAX99_V = []
#GetNumEvents('reco_e',0.05,0.85,NPPRESQ+' and pi0_score > 0.7 and category!=111 and n_showers_contained == 1')

for BDTCUT in BDTCUT_V:
    print ('BDT cut : %.05f'%BDTCUT)
    Q = NPVLCUTQ+" and n_showers_contained==1"#stages_queries[1]
    #print (Q)
    Q += ' and nonpi0_score > %.05f and pi0_score < 1.0 and category != 111'%BDTCUT
    d,e,m68,M68,m95,M95,m99,M99 = GetNumEvents('reco_e',0.15,1.55,Q)
    DATA_V.append(d)
    EXP_V.append(e)
    MIN68_V.append(m68)
    MAX68_V.append(M68)
    MIN95_V.append(m95)
    MAX95_V.append(M95)
    MIN99_V.append(m99)
    MAX99_V.append(M99)
plt.figure(figsize=(10,6))
plt.fill_between(BDTCUT_V,MIN99_V,MAX99_V,alpha=0.2,color='c',label='99% interval')
plt.fill_between(BDTCUT_V,MIN95_V,MAX95_V,alpha=0.2,color='b',label='95% interval')
plt.fill_between(BDTCUT_V,MIN68_V,MAX68_V,alpha=0.2,color='m',label='68% interval')
plt.plot(BDTCUT_V,DATA_V,color='r',lw=2,label='observation')
plt.plot(BDTCUT_V,EXP_V,color='b',lw=2,label='expectation')
plt.grid()
plt.xlabel('$\pi^0$ BDT score cut')
plt.ylabel('surviving events [%.02fE20 POT]'%(pot/1e20))
#plt.gca().ticklabel_format(scilimits=(-2,4))
plt.yscale('log')
plt.legend(loc=1,fontsize=14)
plt.show()


In [ ]:
#cccpi.query(QUERY)[["run","sub","evt","reco_e"]]
tablevars = ["run","sub","evt","reco_e","pi0_score","nonpi0_score"]
tablevars+=TRAINVAR
print(tablevars)
#data.query(PRESQ+" & run==14822")[tablevars]
data.query("run==5151")[tablevars]


In [ ]:
#mc.query(QUERY+"& ccnc==0 & nu_pdg==14 & mcf_np0==0")[["run","sub","evt","reco_e","mcf_lep_e","mcf_mcshr_elec_etot"]]
data.query(QUERY)[["run","sub","evt","reco_e","pi0_score","nonpi0_score","bkg_0p_score","n_tracks_contained","shr_hits_tot","secondshower_Y_nhit","shrmoliereavg","subcluster","trkpid","shr_tkfit_dedx_max","tksh_angle","tksh_distance"]]

In [ ]:
PQUERY = QUERY #PRESQ NPPRESQ NPVLCUTQ NPLCUTQ
PQUERY += ' and reco_e > 0.15 and reco_e < 1.55'
print(PQUERY)
plots = []
plots.append(['tksh_distance',20,(0,40),"trk-shr start 3D distance [cm]"])
#plots.append(['shr_tkfit_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 1-5 cm) [MeV/cm]"])
plots.append(['shr_tkfit_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-4 cm) [MeV/cm]"])
plots.append(['shr_tkfit_dedx_max',20,(0,10),"shr tkfit dE/dx (best-plane, 0-4 cm) [MeV/cm]"])
plots.append(['tksh_angle',20,(-1,1),"cos(tksh angle)"])
plots.append(['trkfit',20,(0,1),"Fraction of Track-fitted points"])
plots.append(['shrmoliereavg',20,(0,50),"average Moliere angle [degrees]"])
plots.append(['trkpid',15,(-1,1),"track LLR PID"])
plots.append(['n_showers_contained',10,(0,10),"n showers contained"])
plots.append(['n_tracks_contained',10,(0,10),"n tracks contained"])
plots.append(['shr_score',20,(0,0.5),"shr score"])
plots.append(['trk_score',20,(0.5,1.0),"trk score"])
plots.append(['hits_ratio',20,(0,1),"shower hits/all hits"])
plots.append(['slclustfrac',20,(0,1),"slice clustered fraction"])
#plots.append(['CosmicIP',20,(0,200),"Cosmic IP from vtx [cm]"])
plots.append(['reco_nu_vtx_sce_x',20,(-10,260),"x"])
plots.append(['reco_nu_vtx_sce_y',20,(-120,120),"y"])
plots.append(['reco_nu_vtx_sce_z',20,(0,1100),"z"])
plots.append(['shr_trk_sce_start_y',20,(-120,120),"shr_trk_sce_start y"])
plots.append(['shr_trk_sce_end_y',20,(-120,120),"shr_trk_sce_end y"])
#plots.append(['reco_e_rqe',20,(0,2),"rqe"])
plots.append(['trkshrhitdist2',20,(0,40),"2D trk-shr distance (Y)"])
plots.append(['subcluster',20,(0,20),"N sub-clusters in shower"])
#plots.append(['shrmoliererms',20,(0,5000),"RMS Moliere angle [degrees]"])
plots.append(['shr_energy_tot_cali',20,(0,1),"shr energy (calibrated) [GeV]"])
plots.append(['trk_energy_tot',20,(0,1),"trk energy (range, P) [GeV]"])
plots.append(['contained_fraction',20,(0.,1),"contained fraction"])
plots.append(['hits_u',20,(0.,1000),"N clustered hits U plane"])
plots.append(['hits_v',20,(0.,1000),"N clustered hits V plane"])
plots.append(['hits_y',20,(0.,1000),"N clustered hits Y plane"])
#plots.append(['pi0_mass_Y',25,(-50.,200),"pi0 mass [Mev]"])
#plots.append(['pi0_gammadot',20,(-1,1),"2-shower angle"])
plots.append(['secondshower_Y_nhit',20,(0,200),"Nhit 2nd shower (Y)"])
plots.append(['secondshower_Y_dot',20,(-1,1),"cos(2nd shower direction wrt vtx) (Y)"])
plots.append(['anglediff_Y',20,(0,350),"angle diff 1st-2nd shower (Y) [degrees]"])
plots.append(['secondshower_Y_vtxdist',20,(0.,200),"vtx dist 2nd shower (Y)"])
plots.append(['CosmicIPAll3D',20,(0,200),"CosmicIPAll3D [cm]"])
plots.append(['CosmicDirAll3D',20,(-1,1),"cos(CosmicDirAll3D)"])
plots.append(['shr_theta',20,(0,4),"shr theta"])
plots.append(['shr_phi',20,(-4,4),"shr phi"])
plots.append(['trk_theta',20,(0,4),"trk theta"])
plots.append(['trk_phi',20,(-4,4),"trk phi"]),
plots.append(['trk_len',20,(0,100),"trk len"])
plots.append(['reco_e', 14, (0.15,1.55), "Reconstructed Energy [GeV]"])
#plots.append(['NeutrinoEnergy0', 20, (0,2000), r"Reconstructed Calorimetric Energy (U) [MeV]"])
#plots.append(['NeutrinoEnergy1', 20, (0,2000), r"Reconstructed Calorimetric Energy (V) [MeV]"])
plots.append(['NeutrinoEnergy2', 20, (0,2000), r"Reconstructed Calorimetric Energy (Y) [MeV]"])
plots.append(['pt',20,(0,2),"pt [GeV]"])
plots.append(['ptOverP',20,(0,1),"pt/p"])
plots.append(['phi1MinusPhi2',13,(-6.5,6.5),"shr phi - trk phi"])
plots.append(['theta1PlusTheta2',13,(0,6.5),"shr theta + trk theta"])
plots.append(['protonenergy',12,(0,0.6),"proton kinetic energy [GeV]"])
plots.append(['Xbj',20,(0,3),"Bjorken X"])
plots.append(['Ybj',20,(0,1),"Bjorken Y"])
plots.append(['Q2',20,(0,1),"Q^2"])
plots.append(['Mhad',20,(0,1),"Mhad (Q0)"])
plots.append(['q3',20,(0,2),"Q3"])
plots.append(['aT',20,(0,1),"#alpha_T"])
#plots.append(['slnhits',20,(0.,5000),"N total slice hits"])
#plots.append(['trk_hits_tot',20,(0.,2000),"Total N hits in tracks"])
#plots.append(['trk_hits_u_tot',20,(0.,700),"Total N hits in tracks (U)"])
#plots.append(['trk_hits_v_tot',20,(0.,700),"Total N hits in tracks (V)"])
#plots.append(['trk_hits_y_tot',20,(0.,700),"Total N hits in tracks (Y)"])
#plots.append(['shr_hits_tot',20,(0.,2000),"Total N hits in showers"])
#plots.append(['shr_hits_u_tot',20,(0.,800),"Total N hits in showers (U)"])
#plots.append(['shr_hits_v_tot',20,(0.,800),"Total N hits in showers (V)"])
#plots.append(['shr_hits_y_tot',20,(0.,800),"Total N hits in showers (Y)"])
#plots.append(['shrsubclusters0',20,(0,20),"N sub-clusters in shower (U)"])
#plots.append(['shrsubclusters1',20,(0,20),"N sub-clusters in shower (V)"])
#plots.append(['shrsubclusters2',20,(0,20),"N sub-clusters in shower (Y)"])
if USEBDT:
    #plots = []
    #plots.append(['pi0_score',20,(0.9,1),"$\pi^0$ BDT response"])
    #plots.append(['nonpi0_score',20,(0.9,1),"non-$\pi^0$ BDT response"])
    plots.append(['pi0_score',10,(0.,1.0),"$\pi^0$ BDT response"])
    plots.append(['nonpi0_score',10,(0.,1.0),"non-$\pi^0$ BDT response"])
    plots.append(['bkg_0p_score',10,(0.,1.0),"Background 1e0p BDT response"])
    #plots.append(['pi0_score',10,(0.,0.5),"$\pi^0$ BDT response"])
    #plots.append(['nonpi0_score',10,(0.,0.5),"non-$\pi^0$ BDT response"])
    #plots.append(['pi0_score',5,(0.5,1.0),"$\pi^0$ BDT response"])
    #plots.append(['nonpi0_score',5,(0.5,1.0),"non-$\pi^0$ BDT response"])
#plots = []
for VARIABLE, BINS, RANGE, XTIT in plots:
    print(VARIABLE, BINS, RANGE, XTIT)
    fig, ax1, ax2 = my_plotter.plot_variable(
        VARIABLE,   
        query=PQUERY,
        kind="event_category",
        #kind="interaction",
        #kind="sample",
        draw_sys=True,
        stacksort=3,
        title=XTIT,
        bins=BINS,
        range=RANGE,
    )[0:3]
    #ax1.set_ylim(0,ax1.get_ylim()[1]*1.4)
    #ax1.set_yscale("log")
    #fig.show()
    #fig.savefig(ls.plots_path+"/run1-5e19_pi0sel_v2/"+VARIABLE+".pdf")
    #fig.savefig(ls.plots_path+""+VARIABLE+"_highscore.pdf")
    #break

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import pandas as pd
import matplotlib.pyplot as plt

BINS = np.linspace(0,1,25)
VAR = 'pi0_score'

nueplot = nue.query(QUERY)
nueplotecut = nueplot.query('nu_e < 0.8')

fig = plt.figure(figsize=(6,6))
plt.hist(nueplot[VAR].values,bins=BINS,histtype='step',lw=2,color='red',label='all nue')
plt.hist(nueplotecut[VAR].values,bins=BINS,histtype='step',lw=2,color='m',label='nue < 0.8 GeV')
#plt.hist(nue[VAR].values,bins=BINS,histtype='step',lw=2,color='green')
plt.grid()
plt.legend()
plt.title(VAR)
plt.show()

BINS2D = (np.linspace(0,3.0,25),BINS)

fig = plt.figure(figsize=(6,6))
plt.hist2d(nueplot['nu_e'].values,nueplot[VAR].values,bins=BINS2D)
#plt.hist(nue[VAR].values,bins=BINS,histtype='step',lw=2,color='green')
plt.grid()
plt.show()

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import pandas as pd
import matplotlib.pyplot as plt

x_range = (0.1,0.8)
n_bins = 7

n_tot = np.empty([50, n_bins])
n_cv_tot = np.empty(n_bins)
n_tot.fill(0)
n_cv_tot.fill(0)
score_cut = 0.997

my_cmap = cm.get_cmap('viridis')

for t in samples:
    if t in ["ext", "data", "lee"]:
        continue
    tree = samples[t]
    
    extra_query = ""
    if t == "mc":
        extra_query = "& ~(nu_pdg == 12 & ccnc == 0) & ~(npi0 != 0 & ccnc == 1)"
        
    queried_tree = tree.query("selected==1 & global_score > %g %s & interaction == 1" % (score_cut, extra_query))
    variable = queried_tree["reco_e"]
    genie_weights = queried_tree["weightsGenie"]
    spline_fix = queried_tree["weightSpline"]*weights[t]

    s = genie_weights
    df = pd.DataFrame(s.values.tolist())


    n_cv, bins = np.histogram(
        variable,
        range=x_range,
        bins=n_bins,
        weights=spline_fix)
    
    n_cv_tot += n_cv

    if not df.empty:
        for i in range(50):
            weight = df[i].values
            weight[np.isnan(weight)] = 1
            weight[weight > 100] = 1

            n, bins = np.histogram(
                variable, weights=weight*spline_fix, range=x_range, bins=n_bins)

            n_tot[i] += n

        
bincenters = 0.5 * (bins[1:] + bins[:-1])

left,right = bins[:-1],bins[1:]
X = np.array([left,right]).T.flatten()
Y = np.array([n_cv_tot,n_cv_tot]).T.flatten()
plt.plot(X,Y,color='r',linewidth=2,label='Central value')

    
cov = np.empty([len(n_cv), len(n_cv)])
cov.fill(0)

my_norm = Normalize(vmin=min((sum(n) for n in n_tot)), vmax=max((sum(n) for n in n_tot)))

for n in n_tot:
    left,right = bins[:-1],bins[1:]
    X = np.array([left,right]).T.flatten()
    Y = np.array([n,n]).T.flatten()
    plt.plot(X,Y,color=my_cmap(my_norm(sum(n))),zorder=-32)
    for i in range(len(n_cv)):
        for j in range(len(n_cv)):
            cov[i][j] += (n[i] - n_cv_tot[i]) * (n[j] - n_cv_tot[j])

cov /= 50
            
frac_cov = np.empty([len(n_cv), len(n_cv)])
corr = np.empty([len(n_cv), len(n_cv)])

for i in range(len(n_cv)):
    for j in range(len(n_cv)):
        frac_cov[i][j] = cov[i][j] / (n_cv_tot[i] * n_cv_tot[j])
        corr[i][j] = cov[i][j] / np.sqrt(cov[i][i] * cov[j][j])

        
plt.errorbar(
    bincenters, 
    n_cv_tot,
    yerr=np.sqrt(np.diag(cov)), 
    fmt='none', 
    ecolor='r',
    linewidth=2,
    label='GENIE sys. uncertainties')

print(np.sqrt(np.diag(cov))/n_cv_tot)
plt.xlim(x_range[0], x_range[1])
plt.xlabel(r"$E_{deposited}$ [GeV]")
plt.legend()
plt.ylim(ymin=0)
plt.tight_layout()
plt.savefig("plots/spec.pdf", transparent=True)

In [ ]:
fig, ax = plt.subplots(1, 1)
pos = ax.imshow(frac_cov, origin='lower', cmap='viridis')
ax.set_title("Fractional covariance matrix")
ax.set_ylabel("Bin number")
ax.set_xlabel("Bin number")
fig.colorbar(pos)
fig.tight_layout()
fig.savefig("plots/frac.pdf", transparent=True)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_title("Correlation matrix")
pos = ax.imshow(corr, origin='lower', cmap='inferno')
ax.set_ylabel("Bin number")
ax.set_xlabel("Bin number")
fig.colorbar(pos)
fig.tight_layout()
fig.savefig("plots/corr.pdf", transparent=True)

In [ ]:
my_plotter.plot_2d(
    "pi0_score",
    "nonpi0_score",
    query="selected==1",
    bins_x=50,
    bins_y=50,
    range_x=(0., 1),
    range_y=(0., 1))